# Fine Tuning Phase als Experiment

In [1]:
import arrow
import learn2learn as l2l
import numpy as np
import os
import pickle
import torch
from torch.nn import Module, Linear, Sequential, ReLU
from torch.nn.functional import mse_loss
from torch.optim import Adam, SGD
from torch.utils.data import TensorDataset
from sklearn.model_selection import train_test_split
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from utils.evalUtils import print_confusion_matrix

from sklearn.preprocessing import MinMaxScaler

In [2]:
%run -i ./scripts/setConfigs.py

Set configs..


## Read Models
### Meta-AE M_1

In [3]:
%run -i ./scripts/ReadSimpleAE_MetaModel.py

Load Meta Model AE..
/Users/torge/Development/master/masterthesis_code/02_Experimente/MetaLearning/models/model_bib/20200319_firstMetaModel.pt
SimpleAutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=17, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=8, bias=True)
    (3): Tanh()
  )
  (decoder): Sequential(
    (0): Linear(in_features=8, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=17, bias=True)
    (3): Tanh()
  )
)


### LogReg Meta

In [4]:
%run -i ./scripts/ReadLogReg_Meta.py

Load trained LogReg..
LogisticRegression(C=1.0, class_weight={1: 2.0}, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=42, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)


/Users/torge/miniconda3/envs/ma_modells/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


## Load & Prepare Data

In [5]:
data_fn = os.path.join(data_path, 'simulation_data_y_2020_2021_reduced.h5')
df_data_train = pd.read_hdf(data_fn, key='df')
print('Shape of X_train data: {}'.format(df_data_train.shape))

Shape of X_train data: (105216, 17)


In [6]:
data_fn = os.path.join(data_path, 'anomalous_data_y_2022_reduced.h5')
df_data_anormal = pd.read_hdf(data_fn, key='df')
print('Shape of X_test data: {}'.format(df_data_anormal.shape))

Shape of X_test data: (35040, 18)


In [7]:
s_labels = df_data_anormal['label']
df_data_anormal.drop('label', axis=1, inplace=True)
print('Shape of X_test data: {}'.format(df_data_anormal.shape))

Shape of X_test data: (35040, 17)


In [8]:
data_fn = os.path.join(data_path, 'drifted_data_y_2023_reduced_more_cos_phi.h5')
df_data_drifted = pd.read_hdf(data_fn, key='df')
print('Shape of X_drifted data: {}'.format(df_data_drifted.shape))

Shape of X_drifted data: (35040, 18)


In [9]:
s_drift_labels_x_drifted = df_data_drifted['drift_labels']
df_data_drifted.drop('drift_labels', axis=1, inplace=True)
print('Shape of X_drifted data: {}'.format(df_data_drifted.shape))

Shape of X_drifted data: (35040, 17)


In [10]:
data_fn = os.path.join(data_path, 'anomalous_drifted_data_y_2023_reduced_more_cos_phi.h5')
df_data_drifted_ano = pd.read_hdf(data_fn, key='df')
print('Shape of X_drifted,ano data: {}'.format(df_data_drifted_ano.shape))

Shape of X_drifted,ano data: (35040, 19)


In [11]:
s_drifted_ano_drift_labels = df_data_drifted_ano['drift_labels']
s_drifted_ano_ano_labels = df_data_drifted_ano['anomaly_labels']

In [12]:
df_data_drifted_ano.drop(['drift_labels', 'anomaly_labels'], axis=1, inplace=True)
print('Shape of X_drifted,ano data: {}'.format(df_data_drifted_ano.shape))

Shape of X_drifted,ano data: (35040, 17)


In [13]:
print('Scale data..')
scaler_train = MinMaxScaler((-1,1))
scaler_train = scaler_train.fit(df_data_train)
scaled_train = scaler_train.transform(df_data_train.to_numpy())
scaled_anormal = scaler_train.transform(df_data_anormal.to_numpy())
scaled_drifted = scaler_train.transform(df_data_drifted.to_numpy())
scaled_drifted_ano = scaler_train.transform(df_data_drifted_ano.to_numpy())

Scale data..


In [14]:
# build tensor from numpy
anormal_torch_tensor = torch.from_numpy(scaled_anormal).type(torch.FloatTensor)
anormal_drifted_torch_tensor = torch.from_numpy(scaled_drifted_ano).type(torch.FloatTensor)
drifted_torch_tensor_X = torch.from_numpy(scaled_drifted).type(torch.FloatTensor)

## Prepare Datasets

In [15]:
# build tensor from numpy
drifted_torch_tensor_y = torch.from_numpy(s_drift_labels_x_drifted.to_numpy().reshape(len(s_drift_labels_x_drifted),1)).type(torch.FloatTensor)
drifted_anormal_torch_tensor_y = torch.from_numpy(s_drifted_ano_drift_labels.to_numpy().reshape(len(s_drifted_ano_drift_labels),1)).type(torch.FloatTensor)
drifted_anormal_torch_tensor_anos_y = torch.from_numpy(s_drifted_ano_ano_labels.to_numpy().reshape(len(s_drifted_ano_ano_labels),1)).type(torch.FloatTensor)

In [16]:
# build pytorch dataset from tensor
drifted_dataset = TensorDataset(drifted_torch_tensor_X, drifted_torch_tensor_y)
drifted_anormal_dataset = torch.utils.data.TensorDataset(anormal_drifted_torch_tensor, drifted_anormal_torch_tensor_y)

### Set Optimizer

In [17]:
optimizer = torch.optim.Adam(meta_model.parameters(), lr=0.001)

## Run Experiments

In [18]:
from Experiments.FineTuneExperiment import FineTuneExperiment

In [19]:
NUM_ITERATIONS = 1000
NUM_EXPERIMENTS = 100
K = 5
FILTERED_CLASSES = [1, 2, 3]

In [20]:
exp = FineTuneExperiment(num_fine_tuner=NUM_EXPERIMENTS, model=meta_model, fine_tune_data=drifted_dataset,
                 eval_data=drifted_anormal_dataset, k=K, fine_tune_iterations=NUM_ITERATIONS,
                 optimizer=optimizer, fine_tune_classes=FILTERED_CLASSES, 
                 classifier=clf_meta, eval_label=drifted_anormal_torch_tensor_anos_y)

exp.run()

Current Tuner: 100%|██████████| 100/100 [31:52<00:00, 19.13s/it] 


In [23]:
exp.avg_kpis()

++++++++++++++++++++++++++++++
Average Accuracy: 85.53872716892538
Average Precision: 33.50977459533905
Average Specifity: 92.06774561534051
Average Sensitivity: 94.96796155348427
++++++++++++++++++++++++++++++
